# Project

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import os

from tqdm import trange
from pathlib import Path
from flygym import Fly, Camera
from flygym.examples.plume_tracking.arena import OdorPlumeArena
from controller import PlumeNavigationTask

In [2]:
# Function to explore the structure of an HDF5 file
def explore_hdf5(file):
    def print_structure(name, obj):
        indent = '  ' * name.count('/')
        if isinstance(obj, h5py.Group):
            print(f"{indent}Group: {name}")
        elif isinstance(obj, h5py.Dataset):
            print(f"{indent}Dataset: {name}, shape: {obj.shape}, dtype: {obj.dtype}")
    
    # Visit all items in the file and apply the print_structure function
    file.visititems(print_structure)

# Open the HDF5 file in read mode and explore its structure
with h5py.File('plume.hdf5', 'r') as f:
    print("Structure of the HDF5 file:")
    explore_hdf5(f)

Structure of the HDF5 file:
Dataset: inflow_pos, shape: (2,), dtype: int64
Dataset: inflow_radius, shape: (), dtype: int64
Dataset: inflow_scaler, shape: (), dtype: float64
Dataset: plume, shape: (18571, 320, 480), dtype: float16


In [3]:

plume_data_path = Path("./plume.hdf5")

# Initialize the OdorPlumeArena
arena = OdorPlumeArena(
    plume_data_path=plume_data_path,
)

In [4]:
# Creating a list of contact sensor placements for each leg and segment using list comprehension.
contact_sensor_placements = [
    f"{leg}{segment}"
    for leg in ["LF", "LM", "LH", "RF", "RM", "RH"]
    for segment in ["Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
]

# Creating a Fly object with specific spawn position, orientation, sensor placements, and other features.
fly = Fly(
    enable_adhesion=True,
    draw_adhesion=True,
    enable_olfaction=True,
    enable_vision=False,
    contact_sensor_placements=contact_sensor_placements,
    spawn_pos=(200, 80, 0.25),
    spawn_orientation=(0, 0, np.pi),
)

# Creating a Camera object associated with the Fly object, setting its ID, playback speed, and window size.
cam = Camera(
    fly=fly,
    camera_id="birdeye_cam",
    play_speed=0.2,
    window_size=(1000, 608),
)

# Creating a PlumeNavigationTask to simulate the fly's navigation within a defined arena, observed by the camera.
sim = PlumeNavigationTask(
    fly=fly,
    cameras=[cam],
    arena=arena,
)

In [5]:
# Reset the simulation and get the initial observation
obs, _ = sim.reset()
HRC_model = []

# Iterate over each step in the plume grid
for i in trange(len(arena.plume_grid)):
    
    # The controller doesn't work, so we decide to not move the fly
    obs = sim.step(np.array([0, 0]))
    
    # Render the simulation and get the rendered image
    sim.render()
    rendered_img = sim.render()[0]
    
    if rendered_img is not None:
        # Calculate the HRC value for the current frame
        hrc_value = HRC_model[-1] if HRC_model else 0

# Create the output directory if it doesn't exist
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# Save the simulation video to the output directory
cam.save_video("outputs/plume.mp4", stabilization_time=0)

  0%|          | 0/18571 [00:00<?, ?it/s]

100%|██████████| 18571/18571 [02:22<00:00, 130.66it/s]
